## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Rikitea,PF,77.23,overcast clouds,-23.1203,-134.9692,People ThankYou
1,1,Butaritari,KI,82.38,light rain,3.0707,172.7902,Isles Sunset Lodge
2,2,Vaini,TO,81.48,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
3,3,Mar Del Plata,AR,82.42,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,4,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  73
What is the maximum temperature you would like for your trip?  88


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Rikitea,PF,77.23,overcast clouds,-23.1203,-134.9692,People ThankYou
1,1,Butaritari,KI,82.38,light rain,3.0707,172.7902,Isles Sunset Lodge
2,2,Vaini,TO,81.48,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
3,3,Mar Del Plata,AR,82.42,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,4,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,5,Atuona,PF,77.63,few clouds,-9.8000,-139.0333,Villa Enata
6,6,Alofi,NU,81.70,broken clouds,-19.0595,-169.9187,Taloa Heights
7,7,Cabo San Lucas,MX,79.52,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
8,8,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,Tanna Lodge
9,9,Buala,SB,82.17,light rain,-8.1450,159.5921,Maringe Lagoon Lodge


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                161
City                   161
Country                161
Max Temp               161
Current Description    161
Lat                    161
Lng                    161
Hotel Name             161
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.copy()
clean_df.dropna(how='any', inplace=True)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.23,overcast clouds,-23.1203,-134.9692,
1,Butaritari,KI,82.38,light rain,3.0707,172.7902,
2,Vaini,TO,81.48,overcast clouds,-21.2000,-175.2000,
3,Mar Del Plata,AR,82.42,clear sky,-38.0023,-57.5575,
4,Kapaa,US,80.58,clear sky,22.0752,-159.3190,
5,Atuona,PF,77.63,few clouds,-9.8000,-139.0333,
6,Alofi,NU,81.70,broken clouds,-19.0595,-169.9187,
7,Cabo San Lucas,MX,79.52,clear sky,22.8909,-109.9124,
8,Isangel,VU,77.11,heavy intensity rain,-19.5500,169.2667,
9,Buala,SB,82.17,light rain,-8.1450,159.5921,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.23,overcast clouds,-23.1203,-134.9692,People ThankYou
1,Butaritari,KI,82.38,light rain,3.0707,172.7902,Isles Sunset Lodge
2,Vaini,TO,81.48,overcast clouds,-21.2000,-175.2000,Keleti Beach Resort
3,Mar Del Plata,AR,82.42,clear sky,-38.0023,-57.5575,Gran Hotel Mar del Plata
4,Kapaa,US,80.58,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
156,Saint-Paul,RE,77.09,broken clouds,-21.0096,55.2707,Boucan Canot
157,Mount Isa,AU,76.77,broken clouds,-20.7333,139.5000,ibis Styles Mt Isa Verona
158,Alice Springs,AU,76.55,overcast clouds,-23.7000,133.8833,Desert Palms Alice Springs
159,Villa Maria,AR,80.82,clear sky,-32.4075,-63.2402,Hotel Milenium


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
print(fig)

Figure(layout=FigureLayout(height='420px'))
